## [Project: Plotting GDP Data on a World Map - Part 1]

In [1]:
PATH = '/content/drive/MyDrive/etc/isp_gdp_csv_files/'
!pip install pygal_maps_world

     |████████████████████████████████| 270 kB 14.4 MB/s 
     |████████████████████████████████| 127 kB 27.6 MB/s 
  Created wheel for pygal-maps-world: filename=pygal_maps_world-1.0.2-py3-none-any.whl size=278561 sha256=48fbdd56bb9bb24aa22a22cfd5c9bd56d296a0a3c775162028d3162c28b52d05
  Stored in directory: /root/.cache/pip/wheels/54/e6/11/5be0d3206bdc0ea8f0fcf1fe32661d7e614863c8b6a22655ae
Successfully built pygal-maps-world


In [18]:
import pygal
import pygal_maps_world

In [19]:
from pygal_maps_world.maps import World

In [17]:
!pip install pygal_maps_world

In [21]:
"""
Project for Week 3 of "Python Data Visualization".
Unify data via common country name.

Be sure to read the project description page for further information
about the expected behavior of the program.
"""

import csv
import math
import pygal

def read_csv_as_nested_dict(filename, keyfield, separator, quote):
    """
    Inputs:
      filename  - Name of CSV file
      keyfield  - Field to use as key for rows
      separator - Character that separates fields
      quote     - Character used to optionally quote fields

    Output:
      Returns a dictionary of dictionaries where the outer dictionary
      maps the value in the key_field to the corresponding row in the
      CSV file.  The inner dictionaries map the field names to the
      field values for that row.
    """

    table = {}
    with open(filename, mode='rt', newline='') as csv_file:
        csv_reader = csv.DictReader(csv_file,
                                    # skipinitialspace = True,
                                    delimiter = separator,
                                    quotechar = quote)
        for row in csv_reader:
            table[row[keyfield]] = row
    return table


def reconcile_countries_by_name(plot_countries, gdp_countries):
    """
    Inputs:
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      gdp_countries  - Dictionary whose keys are country names used in GDP data

    Output:
      A tuple containing a dictionary and a set.  The dictionary maps
      country codes from plot_countries to country names from
      gdp_countries The set contains the country codes from
      plot_countries that were not found in gdp_countries.
    """
    return_dict = {}
    return_set = set()

    for code, name in plot_countries.items():
        if name in gdp_countries.keys():
            return_dict[code] = name
        else:
            return_set.add(code)

    return (return_dict, return_set) 
    


def build_map_dict_by_name(gdpinfo, plot_countries, year):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      year           - String year to create GDP mapping for

    Output:
      A tuple containing a dictionary and two sets.  The dictionary
      maps country codes from plot_countries to the log (base 10) of
      the GDP value for that country in the specified year.  The first
      set contains the country codes from plot_countries that were not
      found in the GDP data file.  The second set contains the country
      codes from plot_countries that were found in the GDP data file, but
      have no GDP data for the specified year.
    """
    gdp_data = read_csv_as_nested_dict(gdpinfo['gdpfile'],
                                       gdpinfo['country_name'],
                                       gdpinfo['separator'],
                                       gdpinfo['quote'])
    
    reconciled_tuple = reconcile_countries_by_name(plot_countries, gdp_data)

    return_dict = {}
    return_set1 = reconciled_tuple[1]
    return_set2 = set() 

    for code, name in reconciled_tuple[0].items():
        if gdp_data[name][year] != '':
            return_dict[code] = math.log10(float(gdp_data[name][year]))
        else:
            return_set2.add(code)

    return (return_dict, return_set1, return_set2)


def render_world_map(gdpinfo, plot_countries, year, map_file):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      year           - String year to create GDP mapping for
      map_file       - Name of output file to create

    Output:
      Returns None.

    Action:
      Creates a world map plot of the GDP data for the given year and
      writes it to a file named by map_file.
    """
    dicts = build_map_dict_by_name(gdpinfo, plot_countries, year)
    worldmap_chart = pygal.maps.world.World()
    worldmap_chart.title = 'GDP in '+year
    worldmap_chart.add('GDP presented',dicts[0])
    worldmap_chart.add('No data', dicts[1])
    worldmap_chart.add('No data in '+year, dicts[2])
    worldmap_chart.render_to_file(map_file)


def test_render_world_map():
    """
    Test the project code for several years.
    """
    gdpinfo = {
        "gdpfile": "isp_gdp.csv",
        "separator": ",",
        "quote": '"',
        "min_year": 1960,
        "max_year": 2015,
        "country_name": "Country Name",
        "country_code": "Country Code"
    }

    # Get pygal country code map
    pygal_countries = pygal.maps.world.COUNTRIES

    # 1960
    render_world_map(gdpinfo, pygal_countries, "1960", "isp_gdp_world_name_1960.svg")

    # 1980
    render_world_map(gdpinfo, pygal_countries, "1980", "isp_gdp_world_name_1980.svg")

    # 2000
    render_world_map(gdpinfo, pygal_countries, "2000", "isp_gdp_world_name_2000.svg")

    # 2010
    render_world_map(gdpinfo, pygal_countries, "2010", "isp_gdp_world_name_2010.svg")


# Make sure the following call to test_render_world_map is commented
# out when submitting to OwlTest/CourseraTest.

# test_render_world_map()




## [Project: Plotting GDP Data on a World Map - Part 2]

In [ ]:
"""
Project for Week 4 of "Python Data Visualization".
Unify data via common country codes.

Be sure to read the project description page for further information
about the expected behavior of the program.
"""

import csv
import math
import pygal

def read_csv_as_list(filename, separator, quote):
    """
    Inputs:
      filename  - Name of CSV file
      keyfield  - Field to use as key for rows
      separator - Character that separates fields
      quote     - Character used to optionally quote fields

    Output:
      Returns a dictionary of dictionaries where the outer dictionary
      maps the value in the key_field to the corresponding row in the
      CSV file.  The inner dictionaries map the field names to the
      field values for that row.
    """

    table = []
    with open(filename, mode='rt', newline='') as csv_file:
        csv_reader = csv.DictReader(csv_file,
                                    # skipinitialspace = True,
                                    delimiter = separator,
                                    quotechar = quote)
        for row in csv_reader:
            table.append(row)
    return table


def read_csv_as_nested_dict(filename, keyfield, separator, quote):
    """
    Inputs:
      filename  - Name of CSV file
      keyfield  - Field to use as key for rows
      separator - Character that separates fields
      quote     - Character used to optionally quote fields

    Output:
      Returns a dictionary of dictionaries where the outer dictionary
      maps the value in the key_field to the corresponding row in the
      CSV file.  The inner dictionaries map the field names to the
      field values for that row.
    """

    table = {}
    with open(filename, mode='rt', newline='') as csv_file:
        csv_reader = csv.DictReader(csv_file,
                                    # skipinitialspace = True,
                                    delimiter = separator,
                                    quotechar = quote)
        for row in csv_reader:
            table[row[keyfield]] = row
    return table
    
def build_country_code_converter(codeinfo):
    """
    Inputs:
      codeinfo      - A country code information dictionary

    Output:
      A dictionary whose keys are plot country codes and values
      are world bank country codes, where the code fields in the
      code file are specified in codeinfo.
    """
    return_dict = {}
    code_data = read_csv_as_list(codeinfo['codefile'], codeinfo['separator'], codeinfo['quote'])
            

    for item in code_data:
        return_dict[item[codeinfo['plot_codes']]] = item[codeinfo['data_codes']]
    return return_dict


def reconcile_countries_by_code(codeinfo, plot_countries, gdp_countries):
    """
    Inputs:
      codeinfo       - A country code information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      gdp_countries  - Dictionary whose keys are country codes used in GDP data

    Output:
      A tuple containing a dictionary and a set.  The dictionary maps
      country codes from plot_countries to country codes from
      gdp_countries.  The set contains the country codes from
      plot_countries that did not have a country with a corresponding
      code in gdp_countries.

      Note that all codes should be compared in a case-insensitive
      way.  However, the returned dictionary and set should include
      the codes with the exact same case as they have in
      plot_countries and gdp_countries.
    """

    return_dict = {}
    return_set = set()
    converted_code = build_country_code_converter(codeinfo)
    
    for code in plot_countries:
        for plot_code, gdp_code in converted_code.items():
            if code.upper() == plot_code.upper() and gdp_code.upper() in gdp_countries.keys():
                return_dict[code] = gdp_code
            elif code.upper() == plot_code.upper() and gdp_code.upper() not in gdp_countries.keys():
                return_set.add(code)
        if code not in return_dict.keys():
            return_set.add(code)
        

    return (return_dict, return_set)


def build_map_dict_by_code(gdpinfo, codeinfo, plot_countries, year):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      codeinfo       - A country code information dictionary
      plot_countries - Dictionary mapping plot library country codes to country names
      year           - String year for which to create GDP mapping

    Output:
      A tuple containing a dictionary and two sets.  The dictionary
      maps country codes from plot_countries to the log (base 10) of
      the GDP value for that country in the specified year.  The first
      set contains the country codes from plot_countries that were not
      found in the GDP data file.  The second set contains the country
      codes from plot_countries that were found in the GDP data file, but
      have no GDP data for the specified year.
    """
    return_dict = {} # {plot 코드 : 해당 year의 GDP log값}
    return_set1 = set() #GDP에 없는 plot 코드
    return_set2 = set() #GDP에는 있지만 해당 year에 값이 없는 plot 코드

    gdp_data = read_csv_as_nested_dict(gdpinfo['gdpfile'],
                                       gdpinfo['country_code'],
                                       gdpinfo['separator'],
                                       gdpinfo['quote'])  
    
    reconciled_tuple = reconcile_countries_by_code(codeinfo, plot_countries, gdp_data)
    return_set1 = reconciled_tuple[1]

    for plot_code, gdp_code in reconciled_tuple[0].items():
        if gdp_data[gdp_code][year] != '':
            return_dict[plot_code] = math.log10(float(gdp_data[gdp_code][year]))
        else:
            return_set2.add(plot_code)

    return (return_dict, return_set1, return_set2)

def render_world_map(gdpinfo, codeinfo, plot_countries, year, map_file):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      codeinfo       - A country code information dictionary
      plot_countries - Dictionary mapping plot library country codes to country names
      year           - String year of data
      map_file       - String that is the output map file name

    Output:
      Returns None.

    Action:
      Creates a world map plot of the GDP data in gdp_mapping and outputs
      it to a file named by svg_filename.
    """
    dicts = build_map_dict_by_code(gdpinfo, codeinfo, plot_countries, year)
    worldmap_chart = pygal.maps.world.World()
    worldmap_chart.title = 'GDP in '+year
    worldmap_chart.add('GDP presented', dicts[0])
    worldmap_chart.add('No data', dicts[1])
    worldmap_chart.add('No GDP data in '+year, dicts[2])
    worldmap_chart.render_to_file(map_file)


def test_render_world_map():
    """
    Test the project code for several years
    """
    gdpinfo = {
        "gdpfile": "isp_gdp.csv",
        "separator": ",",
        "quote": '"',
        "min_year": 1960,
        "max_year": 2015,
        "country_name": "Country Name",
        "country_code": "Country Code"
    }

    codeinfo = {
        "codefile": "isp_country_codes.csv",
        "separator": ",",
        "quote": '"',
        "plot_codes": "ISO3166-1-Alpha-2",
        "data_codes": "ISO3166-1-Alpha-3"
    }

    # Get pygal country code map
    pygal_countries = pygal.maps.world.COUNTRIES

    # 1960
    render_world_map(gdpinfo, codeinfo, pygal_countries, "1960", "isp_gdp_world_code_1960.svg")

    # 1980
    render_world_map(gdpinfo, codeinfo, pygal_countries, "1980", "isp_gdp_world_code_1980.svg")

    # 2000
    render_world_map(gdpinfo, codeinfo, pygal_countries, "2000", "isp_gdp_world_code_2000.svg")

    # 2010
    render_world_map(gdpinfo, codeinfo, pygal_countries, "2010", "isp_gdp_world_code_2010.svg")


# Make sure the following call to test_render_world_map is commented
# out when submitting to OwlTest/CourseraTest.

#test_render_world_map()
